### Анализ тональности твитов
Создаю переменную tweets, где будут только позитивные и отрицательные сообщения (без имен пользователя и тд)\
Вывожу хвост из 3 данных, чтобы убедиться, что в переменной все твиты

In [1]:
import pandas as pd

tweets_neg = pd.read_csv('negative.csv', delimiter=';', 
                    names=['none1', 'none2', 'none3', 'text', 'none4', 'none5', 
                           'none6', 'none7', 'none8', 'none9', 'none10', 'none11'])
tweets_pos = pd.read_csv('positive.csv', delimiter=';', 
                    names=['none1', 'none2', 'none3', 'text', 'none4', 'none5', 
                           'none6', 'none7', 'none8', 'none9', 'none10', 'none11'])
tweets = pd.DataFrame()
tweets['neg'], tweets['pos'] = tweets_neg['text'], tweets_pos['text']
tweets.tail(3)

,neg,pos
111920,"Вот и в школу, в говно это идти уже надо(","@_music_lover_13 оооо, еще и Оля) какая прелес..."
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...","@GazetaRu_All Ныне картина того,как у ЕС,сжало..."
111922,Такси везет меня на работу. Раздумываю приплат...,"RT @bazzzilio: @VRSoloviev :)) Я все еще жив, ..."


Загрузка русских стоп-слов с помощью библиотеки nltk

In [2]:
import nltk
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")

Обрабатываю сообщения пользователей с помощью регулярных выражений и строки стоп-слов

1. Из каждого сообщения оставляю только русские слова и привожу к нижнему регистру
2. Делаю из строки массив слов, где разделитель на слова служит любое кол-во пробелов
3. Создаю строку так, чтобы проверяемое слово не было стоп-словом и длина была не меньше 2 и нормализую его

Функция norm_word:  
Для начала проверяю есть ли нужное слово в словаре со словами в начальной форме  
Если есть, то возвращаю слово в начальной форму  
Если нет, то с помощью библиотеки pymorphy2 и его анализатора, возвращаю слово в нормальной форме и запомнимаю его в словаре.    Возращаю результат  
Зачем? Потому что если какждый раз вызывать анализатор pymorphy2, то это будет ОЧЕНЬ долго (я за 5 минут не смог дождаться), так как он каждый раз вызывает свой алгоритм и если одно и тоже слово появляется уже в 10 раз, то и время займет в 10 раз больше. А я буду запоминать значение и если попалось тоже слово, то выводить сразу его.

In [3]:
import re
import pymorphy2

norm = {}
morph = pymorphy2.MorphAnalyzer()

def norm_form(word):
    res = norm.get(word)
    if res:
        return res
    res = morph.parse(word)[0].normal_form
    norm[word] = res
    return res


def cleaner_tweets(tweets):
  k = 1
  tempArr = []
  for tweet in tweets['neg']:
    tweet = re.sub(r'[^а-яА-Я]', ' ', tweet.lower())
    tweet = re.split(r'\s+', tweet)
    tweet = ' '.join([norm_form(word) for word in tweet if len(word) > 2 and word not in russian_stopwords])
    tempArr.append(tweet)
    k += 1
    if k % 10000 == 0: print(f"Уже очищено {k} из 22000 слов")
  tweets['negClean'] = pd.Series(tempArr)
  
  tempArr = []  
  for tweet in tweets['pos']:
    tweet = re.sub(r'[^а-яА-Я]', ' ', tweet.lower())
    tweet = tweet.split()
    tweet = ' '.join([norm_form(word) for word in tweet if len(word) > 2 and word not in russian_stopwords])
    tempArr.append(tweet)
    k += 1
    if k % 10000 == 0: print(f"Уже очищено {k} из 22000 слов")
  tweets['posClean'] = pd.Series(tempArr)

  return tweets
    

tweets = cleaner_tweets(tweets)
tweets.tail(2)

Уже очищено 10000 из 22000 слов
Уже очищено 20000 из 22000 слов
Уже очищено 30000 из 22000 слов
Уже очищено 40000 из 22000 слов
Уже очищено 50000 из 22000 слов
Уже очищено 60000 из 22000 слов
Уже очищено 70000 из 22000 слов
Уже очищено 80000 из 22000 слов
Уже очищено 90000 из 22000 слов
Уже очищено 100000 из 22000 слов
Уже очищено 110000 из 22000 слов
Уже очищено 120000 из 22000 слов
Уже очищено 130000 из 22000 слов
Уже очищено 140000 из 22000 слов
Уже очищено 150000 из 22000 слов
Уже очищено 160000 из 22000 слов
Уже очищено 170000 из 22000 слов
Уже очищено 180000 из 22000 слов
Уже очищено 190000 из 22000 слов
Уже очищено 200000 из 22000 слов
Уже очищено 210000 из 22000 слов
Уже очищено 220000 из 22000 слов


,neg,pos,negClean,posClean
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...","@GazetaRu_All Ныне картина того,как у ЕС,сжало...",тауриэль грусть обнять,ныне картина сжаться страх пытаться убежать ро...
111922,Такси везет меня на работу. Раздумываю приплат...,"RT @bazzzilio: @VRSoloviev :)) Я все еще жив, ...",такси везти работа раздумывать приплатить втащ...,живой придурок перестать путать нельсон мандел...


Записываю уже очищенные негативные и позитивные твиты в один массив.  
Оставляю на них метки, что "0" - это негативный твит, а "1" - это позитивный твит.  
Вывожу полученное слияние  
Далее перемешиваю их, чтобы модель смогла обучиться равномерно, иначе первая половина состояла только из отрицательных твитов.  
Вывожу полученное перемешивание.

In [5]:
negClean = pd.DataFrame()
posClean = pd.DataFrame()

negClean['text'], negClean['state'] = tweets['negClean'], [0]*len(tweets['negClean'])
posClean['text'], posClean['state'] = tweets['posClean'], [1]*len(tweets['posClean'])
clean_tweets = pd.concat([negClean, posClean])
print(clean_tweets.head(2), clean_tweets.tail(2))

clean_tweets = clean_tweets.sample(frac=1).reset_index(drop=True)
print(clean_tweets.head(2), clean_tweets.tail(2))

                                                text  state
0  работа полный пиддес каждый закрытие месяц сви...      0
1          коллега сидеть рубиться долбать винд мочь      0                                                      text  state
111921  ныне картина сжаться страх пытаться убежать ро...      1
111922  живой придурок перестать путать нельсон мандел...      1
                                            text  state
0  пздца представлять козёл ахи мочь дать лёгкий      0
1                             быть неплохой пара      1                                                     text  state
223844  печалька увидеть катажина январь прямо плак плак      0
223845      вчера преаод посмотреть миллыр любимый фильм      1


Векторизую данные с помощью CountVectorizer на триграммы

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(3, 3))
tweetsVect = cv.fit_transform(clean_tweets['text'])

Обучающая и тестовая выборка  
70% обучающей и 30% тестовой  
  
Импортирую логистическую регрессию и обучаю на ней модель.  
Далее строю предсказания на тестовой выборке и при сравнении с правдивыми значениями, вывожу classification_report из модуля sklearn.metrics и смотрю на macro avg f1-score что для триграмм составляет 0.47

In [14]:
from sklearn.model_selection import train_test_split


tw_train, tw_test, st_train, st_test = train_test_split(tweetsVect, clean_tweets['state'], test_size = 0.3)

import numpy as np
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter = 350)
model.fit(tw_train, st_train)

st_pred = model.predict(tw_test)

from sklearn.metrics import classification_report
print(classification_report(st_test, st_pred))

              precision    recall  f1-score   support

           0       0.53      0.97      0.69     33613
           1       0.82      0.15      0.26     33541

    accuracy                           0.56     67154
   macro avg       0.68      0.56      0.47     67154
weighted avg       0.68      0.56      0.47     67154



Векторизую данные с помощью TfidfVectorizer на триграммы

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfdv = CountVectorizer(ngram_range=(3, 3))
tweetsVect = tfdv.fit_transform(clean_tweets['text'])

In [16]:
from sklearn.model_selection import train_test_split


tw_train, tw_test, st_train, st_test = train_test_split(tweetsVect, clean_tweets['state'], test_size = 0.3)

import numpy as np
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter = 350)
model.fit(tw_train, st_train)

st_pred = model.predict(tw_test)

from sklearn.metrics import classification_report
print(classification_report(st_test, st_pred))

              precision    recall  f1-score   support

           0       0.54      0.97      0.69     33836
           1       0.83      0.15      0.26     33318

    accuracy                           0.56     67154
   macro avg       0.68      0.56      0.48     67154
weighted avg       0.68      0.56      0.48     67154

